In [1]:
import os
import tensorflow as tf
import re

import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

import gc
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [51]:
train = pd.read_csv("train 2.csv")
test = pd.read_csv("test 2.csv")
test2 = pd.read_csv("test 2.csv")

In [3]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [5]:
train.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [6]:
test.isnull().sum()

id              0
comment_text    0
dtype: int64

In [7]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_train = train["comment_text"]
list_sentences_test = test["comment_text"]

In [8]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [9]:
maxlen = 300
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [10]:
word_index = tokenizer.word_index

In [11]:
embeddings_index = {}
f = open("../../TextFinder_wc/crawl-300d-2M.vec")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 2000000 word vectors.


In [12]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [15]:
embedding_layer = Embedding(len(word_index) + 1,
                           300,
                           weights=[embedding_matrix],
                           input_length=maxlen,
                           trainable=True)

In [16]:
sequence_input = Input(shape=(300,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = LSTM(60, return_sequences=True, name='lstm_layer')(embedded_sequences)
x = Bidirectional(LSTM(60, return_sequences=True,name='lstm_layer',dropout=0.1,recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)

model = Model(sequence_input, x)
model.compile(loss="binary_crossentropy",
             optimizer='adam',
             metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 300, 300)          63101400  
_________________________________________________________________
lstm_layer (LSTM)            (None, 300, 60)           86640     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300, 120)          58080     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 120)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                6050      
__________

In [25]:
batch_size = 350
epochs = 2
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 1268s 9ms/step - loss: 0.0447 - acc: 0.9833 - val_loss: 0.0473 - val_acc: 0.9823
Epoch 2/2
143613/143613 [==============================] - 1273s 9ms/step - loss: 0.0404 - acc: 0.9845 - val_loss: 0.0479 - val_acc: 0.9816


In [29]:
y_pred = model.predict(X_te)

In [31]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [57]:
test = test.drop("comment_text", axis=1)

In [58]:
test.head()

,id
0,00001cee341fdb12
1,0000247867823ef7
2,00013b17ad220c46
3,00017563c3f7919a
4,00017695ad8997eb


In [68]:
y_pred.shape

(153164, 6)

In [69]:
test['id'].shape

(153164,)

In [71]:
test.list_classes = y_pred

In [73]:
df = pd.read_csv("test_2.csv")

In [76]:
df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred

In [77]:
df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.994303,0.519864,0.956276,0.103586,0.879186,0.297969
1,0000247867823ef7,0.000428,0.000002,0.000044,0.000007,0.000069,0.000009
2,00013b17ad220c46,0.002672,0.000028,0.000269,0.000099,0.000449,0.000096
3,00017563c3f7919a,0.000597,0.000002,0.000061,0.000009,0.000092,0.000013
4,00017695ad8997eb,0.000925,0.000004,0.000080,0.000015,0.000130,0.000019


In [79]:
df.to_csv("subm1.csv")

In [80]:
df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred/1.4

In [81]:
df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.710216,0.371331,0.683054,0.073990,0.627990,0.212835
1,0000247867823ef7,0.000306,0.000001,0.000031,0.000005,0.000049,0.000006
2,00013b17ad220c46,0.001908,0.000020,0.000192,0.000071,0.000321,0.000069
3,00017563c3f7919a,0.000426,0.000001,0.000044,0.000006,0.000066,0.000009
4,00017695ad8997eb,0.000660,0.000003,0.000057,0.000011,0.000093,0.000014


In [82]:
df.to_csv("subm2.csv")

In [83]:
df.shape

(153164, 7)

In [84]:
df.loc[19797]

id               213296d90445e53a
toxic                    0.186355
severe_toxic           0.00083136
obscene                0.00808538
threat                  0.0029572
insult                  0.0197614
identity_hate          0.00788822
Name: 19797, dtype: object

In [86]:
df.duplicated().sum()

0